In [1]:
import requests
import json

# Function to fetch vectors from Qdrant
def fetch_vectors_from_qdrant(collection_name, limit=5000):
    url = f"http://localhost:6333/collections/{collection_name}/points/scroll"
    
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
    }

    body = {
        "limit": limit,
        "with_vector": True,
        "with_payload": True
    }

    response = requests.post(url, headers=headers, data=json.dumps(body))
    if response.status_code == 200:
        return response.json()["result"]["points"]
    else:
        raise Exception(f"Failed to fetch data from Qdrant: {response.text}")

# Fetch the vectors from Qdrant
collection_name = "image_embeddings_resnet34"
points = fetch_vectors_from_qdrant(collection_name, limit=101000)

# Example: points structure
# [
#   {
#     "id": 1,
#     "vector": [0.1, 0.2, 0.3, ...],
#     "payload": {
#       "filename": "image1.jpg",
#       "class": "cat"
#     }
#   },
#   ...
# ]


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorboard.plugins import projector

# Set up TensorBoard log directory
log_dir = './logs/image_embeddings_resnet34/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Prepare the vectors and labels (metadata)
vectors = []
metadata = []

for point in points:
    vectors.append(point['vector'])
    metadata.append(point['payload'].get('class', 'unknown'))

# Convert to numpy array for TensorFlow
vectors_np = np.array(vectors)

# Save Vectors as TensorFlow Variable
embedding_var = tf.Variable(vectors_np, name='embedding')

# Save Labels in metadata.tsv file
metadata_path = os.path.join(log_dir, 'metadata.tsv')
with open(metadata_path, 'w') as f:
    for label in metadata:
        f.write(f"{label}\n")

# Create a checkpoint to store embeddings
checkpoint = tf.train.Checkpoint(embedding=embedding_var)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up TensorBoard Projector config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'

# Save the projector config
projector.visualize_embeddings(log_dir, config)


2024-10-15 14:16:47.835856: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-15 14:16:47.846218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 14:16:47.858367: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 14:16:47.861596: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 14:16:47.870680: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
!tensorboard --logdir=./logs/image_embeddings_resnet34/


2024-10-15 14:16:50.396323: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-15 14:16:50.406206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 14:16:50.418523: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 14:16:50.422323: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 14:16:50.430562: I tensorflow/core/platform/cpu_feature_guar

In [28]:
!pip install tensorflow

  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-4.25.5-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached wrapt-1.16.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached tensorboard-2.17.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.6.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached markdown_it_py-3.0.